In [1]:
import pandas as pd
dataset = pd.read_csv('insurance_pre.csv')

In [2]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
dataset = pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [5]:
independent_variable=dataset[['age','bmi','children','sex_male','smoker_yes']]

In [6]:
dependent_variable=dataset[['charges']]

In [7]:
independent_variable

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [8]:
dependent_variable

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [9]:
!pip install lightgbm

In [10]:
import lightgbm as lg

In [11]:
from lightgbm import LGBMRegressor

In [12]:
from sklearn.metrics import r2_score

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
param_grid={'n_estimators':[50,100],
           'boosting_type':['dart','gbdt','rf'],
           'importance_split':['split','gain'],
           'num_leaves':[50,100]}
grid = GridSearchCV(lg.LGBMRegressor(),param_grid,n_jobs=-1,verbose=3,refit=True)
grid.fit(independent_variable,dependent_variable)
re=grid.cv_results_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\rajes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\rajes\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\rajes\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 1189, in fit
    super().fit(
  File "C:\Users\rajes\anaconda3\lib\site-packages\lightgbm\sklearn.py", line 955, in fit
    self._Booster = train(
  File "C:\Users\rajes\anaconda3\lib\site-packages\lightgbm\engine.py", line 282, in

[LightGBM] [Warning] Unknown parameter: importance_split
[LightGBM] [Warning] Unknown parameter: importance_split
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000093 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1338, number of used features: 5
[LightGBM] [Info] Start training from score 13270.422260
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [14]:
re=grid.cv_results_
table = pd.DataFrame.from_dict(re)

In [15]:
print('Best R_Score value',format(grid.best_params_))
grid_pred = grid.predict(independent_variable)
from sklearn.metrics import r2_score
r_score = r2_score(dependent_variable,grid_pred)
r_score

Best R_Score value {'boosting_type': 'gbdt', 'importance_split': 'split', 'n_estimators': 50, 'num_leaves': 50}
[LightGBM] [Warning] Unknown parameter: importance_split


0.918772819842458

In [16]:
#import pickle
#filename='exponentialRegression.sav'
#output = pickle.dump(regressor,open(filename,'wb'))
#loaded_model = pickle.load(open(filename,'rb'))
#result =loaded_model.predict([[19,27.900,0,0,1]])
#result